In [1]:
import os
import numpy as np
import neuroglancer
from skimage import io
import matplotlib
from pprint import pprint
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2
from tifffile import imwrite

In [2]:
animal = 'DK52'
filename = 'allen_10um_coronal.tiff'
DATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
filepath = os.path.join(DATA, filename)
volume = io.imread(filepath)
print(f'Volume shape={volume.shape} dtype={volume.dtype}')

Volume shape=(1320, 800, 1140) dtype=uint16


In [3]:
sagittal = np.swapaxes(volume, 0,2)
#sagittal = np.rot90(sagittal, axes=(1, 0))
#sagittal = np.flip(sagittal, axis=1)
outpath = os.path.join(DATA, 'allen_10um_sagittal.tiff')
print(sagittal.shape)
imwrite(outpath, sagittal)

(1140, 800, 1320)


In [ ]:
arr = sagittal
midpoint = arr.shape[2] // 2
limit = 10
start = midpoint - 10
end = midpoint + 10
for i in tqdm(range(start, end, 4)):
    plt.title('Z {}'.format(i))
    img = arr[:,:,i]
    #img = np.rot90(img, 1, axes=(1,0))
    #img = np.flip(img, axis=1)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
image_stack = np.zeros(volume.shape)
file_list = []

for i in tqdm(range(0, volume.shape[2], 1)):
    img = volume[:,:,i]
    img = np.rot90(img, 1, axes=(1,0))
    img = np.flip(img, axis=1)
    filepath = os.path.join(OUTPUT, str(i).zfill(3) + '.tif')
    cv2.imwrite(filepath, img)

In [ ]:
# The default atlas created by brainreg is isotropic 25um
#vm2 = np.swapaxes(volume, 0, 2)
#vm2 = np.flip(vm2, axis=0)
vm2 = volume
print(f'new volume shape={vm2.shape}')
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='um',
    scales=[25,25,25])
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.dimensions = dimensions
    s.layout = '4panel'
    s.layers['Allen'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(
            data=vm2,
            dimensions=dimensions,
            voxel_offset=(0, 0, 0),
        ))

print(viewer)